In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [3]:

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
# 
# future thinking here...
# ..disk approach => return the location of X, y on disk instead 

In [4]:
workdir = fu.make_work_dir(); print(workdir)
fu.log(workdir, 'Starting')

/opt/program/artifacts/2020-06-26T142154Z


In [7]:
# place a small section of the pre-processed data into a target file..
x_outfile = f'{workdir}/x_transformed.csv'
y_outfile = f'{workdir}/y_enc.csv'

X_transformed, y_enc, proc_dict = pv2.preprocess(
        X[:1000], y[:1000], neighborhoods, labeled=True)

100%|██████████| 10/10 [00:00<00:00, 359.14it/s]


In [11]:
np.savetxt(x_outfile, X_transformed, delimiter=',')

In [16]:
(X_transformed.shape, y_enc.shape, 
 np.hstack((np.resize(y_enc, (1000, 1)), X_transformed)).shape)

((1000, 85), (1000,), (1000, 86))

In [18]:
both_outfile = f'{workdir}/minidata.csv'
yx_data = np.hstack((np.resize(y_enc, (1000, 1)), X_transformed))
np.savetxt(both_outfile, yx_data, delimiter=',', fmt='%u')

In [20]:
dmatrix = xgb.DMatrix(
    f'{both_outfile}?format=csv&label_column=0&delimiter=,')


[15:28:46] 1000x85 matrix with 85000 entries loaded from /opt/program/artifacts/2020-06-26T142154Z/minidata.csv?format=csv&label_column=0&delimiter=,


In [21]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob')
xgb_model.fit(dmatrix,  verbose=True)

TypeError: fit() missing 1 required positional argument: 'y'

#### references
- [this](https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/data_utils.py) is a handy reference around xgb utils